# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [67]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from openai import AzureOpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [68]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
# openai = OpenAI()

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [69]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('AZURE_OPENAI_API_KEY')
endpoint = os.getenv('ENDPOINT')
version = os.getenv('VERSION')
MODEL = os.getenv('DEPLOYMENT')

client = AzureOpenAI(
    azure_endpoint=endpoint, 
    api_key=api_key,
    api_version=version
)

In [70]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with Selenium for dynamic websites.
    """

    def __init__(self, url):
        self.url = url

        # Set up Selenium WebDriver with Chrome
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run in headless mode
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # Load the webpage
        driver.get(url)
        time.sleep(5)  # Wait for the page to load completely

        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()  # Close the browser

        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [45]:
ed = Website("https://edwarddonner.com")
ed.links

['#wp--skip-link--target',
 'https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwar

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [71]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [47]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [72]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [49]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#wp--skip-link--target
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwardd

In [73]:
def get_links(url):
    website = Website(url)
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [51]:
get_links("https://anthropic.com")

{'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'},
  {'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'},
  {'type': 'transparency page',
   'url': 'https://www.anthropic.com/transparency'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}

In [52]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nvidia/parakeet-tdt-0.6b-v2',
 '/ACE-Step/ACE-Step-v1-3.5B',
 '/nari-labs/Dia-1.6B',
 '/Lightricks/LTX-Video',
 '/Qwen/Qwen3-235B-A22B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/RiverZ/ICEdit',
 '/spaces/smolagents/computer-agent',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/ACE-Step/ACE-Step',
 '/spaces',
 '/datasets/nvidia/Nemotron-CrossThink',
 '/datasets/DMindAI/DMind_Benchmark',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/

In [53]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion page',
   'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [74]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [55]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'about page', 'url': 'https://www.anthropic.com/team'}, {'type': 'education page', 'url': 'https://www.anthropic.com/education'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Team plan
Enterprise plan
Education plan
Explore pricing
Download apps
Claude log in
News
Claude’s character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Le

In [56]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
B

In [75]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [76]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [77]:
get_brochure_user_prompt("anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}]}


"You are looking at a company called: anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nTeam plan\nEnterprise plan\nEducation plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude 3.7 Sonnet\nClaude 3.5 Haiku\nClaude 3 Opus\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nISO\xa042001 certification

In [60]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnvidia/parakeet-tdt-0.6b-v2\nUpdated\n9 days ago\n•\n42.1k\n•\n657\nACE-Step/ACE-Step-v1-3.5B\nUpdated\nabout 7 hours ago\n•\n286\nnari-labs/Dia-1.6B\nUpdated\n4 days ago\n•\n144k\n•\n2.01k\nLightricks/LTX-Video\nUpdated\n4 days ago\n•\n214k\n•\n1.35k\nQwen/Qwen3-235B-A22B\nUpdated\n9 days ago\n•\n65.5k\n•\n758\nBrowse 1M+ models\nSpaces\nRunning\n6.27k\n6.27k\nDeepSite\n🐳\nGenerate any applic

In [78]:
def create_brochure(company_name, url):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [62]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'about page', 'url': 'https://www.anthropic.com/about'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


# Welcome to Anthropic: Where AI Meets Safety, and Laughs Ensure You Don't Cry

---

## About Us

At Anthropic, our mission is simple: build AI that keeps both humanity *and* those pesky robots in check. Founded on a planet where knowledge is practically a superpower, we’ve decided to create machines that won't just help—*they’ll care* (well, sort of). Think of us as the superheroes of AI, only with fewer capes and more coding.

---

## Meet Our Star Player: Claude!

Claude is our star AI model, and he's not just smart; he’s *Claude 3.7 Sonnet*! We’ve trained him to be incredibly intelligent—but fear not! Claude is much better at holding conversations than plotting world domination. So, if you need a chat or want to start a karaoke night, Claude is your go-to AI buddy.

---

## Company Culture: No Robots Allowed (Except Claude)

At Anthropic, we pride ourselves on a culture that’s as innovative as Claude’s poetic side. Here's what you can expect when you join our crew:

- **Transparency, Not Just for Glass Windows**: We believe in honest conversations about responsible AI development—no sugar-coating here!
  
- **Casual Fridays Every Day**: Who wears suits when you’ve got a state-of-the-art AI to hang out with?
  
- **Learning Never Stops**: Join our *Anthropic Academy*! We’re all about growth, and yes, we promise you’ll pick up some cool skills along the way (just maybe leave your villain laughs at home).

---

## Our Customers: Friendly Faces, Not Data-Extractors

Our customers range from curious students to enterprises looking to harness the power of AI. We help them chat with Claude, create effective AI agents, and code like a pro—because what’s more fun than coding with a friend who actually understands you?

Not to brag, but they love our customer support almost as much as they love getting the best out of Claude. We even have case studies to prove it—trust us, they’re not just for show!

---

## Careers: Want to Join the Fun?

We’re always on the lookout for passionate innovators who want to build the future of AI—and keep it safe! If you’re ready to test your skills while keeping society’s well-being in mind, check out our open roles!

You’d fit right in if you can:

- Make Claude giggle (don’t worry, he has a great sense of humor).
- Share our vision for a responsible and safe future for AI.
- Laugh at our bad puns (or come armed with better ones).

---

### Join Us!

If you see yourself chatting with Claude, coding brilliant AI-powered applications, or simply enjoying a great company culture while making a difference, **we want to hear from you!**

Let’s build a safer, more intelligent future together—one joke at a time!

**[Explore Careers at Anthropic](#)**

---

**Anthropic: Where AI Meets Heart…and a Little Bit of Wit!**

In [63]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Welcome to the Hugg-iverse! 🌌

### Hugging Face – Where AI Gets a Hug and a High-Five!

**Are you ready to embark on a journey through the futuristic realms of AI?** 🦾 Here at Hugging Face, we’re not just building the future; we’re throwing an AI party, and everyone’s invited! 🎉

---

## Company Culture: Join the AI Hug Train 🚂💕

At Hugging Face, our culture is as warm and fuzzy as a big bear hug (with just a touch of data science magic). Here’s what to expect when you hop on board:

- **Collaboration is Key**: Whether you’re a seasoned data wizard or an ML newbie, we believe great things happen when minds come together! Expect brainstorming sessions that can turn even the grumpiest dataset into a smile! 😄
  
- **Open Source Love**: We’re all about sharing! Think of us as the ultimate AI potluck where everyone brings their best models and datasets. No “bring your own chips” nonsense here!

- **Work Hard, Play Harder**: We do serious stuff… like serious fun! Participate in AI hackathons, game nights, and maybe even the occasional robot dance-off! 🤖

---

## Meet Our Customers: AI Enthusiasts Assemble! 🦸‍♂️🦸‍♀️

What do Amazon, Google, Microsoft, and Grammarly have in common? They’re all teaming up with Hugging Face! 🌟 With over 50,000 organizations using our platform, we’re practically the Avengers of the AI world. 

- **From start-ups to Fortune 500s**, our customers are on a mission to harness the power of AI for good (and maybe a little mischief too!).

---

## Careers at Hugging Face: The Future Needs YOU! 🚀 

Are you looking for a career that combines passion with creativity? Join us, and let's reshape the AI landscape together! 

- **Jobs Available**: From Machine Learning Engineers who wield the power of models like Thor (but with data), to Community Managers who keep the Hugging Face family connected and thriving. 💪

- **Why Work with Us?** Because waking up to do what you love doesn’t just feel productive, it feels *huggable*! 

- **Remote-friendly**: Work from anywhere you feel cozy—whether it’s your home office or a tropical beach (don’t forget the sunscreen!).

---

## Hugging Face Features: What's Cooking in Our Cozy Kitchen? 🍳

- **Models galore**: Explore over **1 million models**! Whether you're into text generation or video magic, we've got tools for every digital chef! 🍔

- **Datasets for Days**: With 250k+ datasets, you can rub shoulders with the best data in town—no awkward small talk required.

- **Hugging Chat**: Experience the magic of AI chats powered by our innovative tech! It's like having a friendly robot right in your pocket—without the batteries dying on you! 🔋

---

## Ready to Join the Hugging Face Family? 🚀

Sign up today and let’s build the future together—one hug at a time! Because remember, in the world of AI, a little hug can go a long way! 💖

---

**Connect with us:** 🌐

- **GitHub**: [huggingface.co](https://github.com/huggingface)  
- **Twitter**: [Hugging Face](https://twitter.com/huggingface)  
- **LinkedIn**: [Hugging Face](https://linkedin.com/company/hugging-face)  
- **Discord**: Enter the happy zone: [Join Us!](https://huggingface.co/discord)

---

*Hugging Face – The AI community building the future, one model at a time!* 

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [79]:
def stream_brochure(company_name, url):
    stream = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        try:
            # Access attributes directly instead of using .get()
            delta_content = chunk.choices[0].delta.content if chunk.choices else ""
            if delta_content:
                response += delta_content
                response = response.replace("```", "").replace("markdown", "")
                update_display(Markdown(response), display_id=display_handle.display_id)
        except (IndexError, AttributeError) as e:
            print(f"⚠️ Error processing chunk: {e}")

    return response

In [65]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'learning page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}]}


# Welcome to Anthropic: Where AI Meets Safety with a Touch of Sass! 🎉

## Who Are We? 
At **Anthropic**, we create dynamic AI that serves humanity and put safety right at the forefront. Think of us as the dandy safety patrol officers of the AI world with a very clever, chatty companion named **Claude**! Whether you need support for coding, collaboration, or those downright tricky customer queries, Claude is here to serve up answers like a master chef—just without the cooking!

## Meet Claude! 🤖
Claude is our enchanting AI model that has gone through more evolutions than a Pokémon! Currently rocking the *3.7 Sonnet* version (take that, Shakespeare!), Claude can help with everything from research to product design. Plus, he’ll keep it safe while doing so, because let's face it, no one wants a rogue AI causing chaos in the digital library!

## Company Culture: A Safe Haven for Big Thinkers 🧠✨
At Anthropic, we believe in **bold steps forward** mixed with intentional pauses. It’s like taking a leap into the future while holding the hand of safety! We encourage *deep thinking* (cue: contemplative philosopher pose) and cater to individuals who have both brains and heart. 

### Our Hot Takes on AI Safety: 
- If knowledge is power, let’s build some *well-behaved* machines with all that power.
- We prioritize **transparency** and a **responsible scaling policy** that even your grandma would approve of.

## Careers: Join Our AI-ssential Team! 🏢💼
Feeling the call to help us shape the safe AI landscape? Check out our open roles! We're looking for passionate individuals who want to contribute to a world where AI serves humanity's long-term well-being; a bit like superheroes, but in the digital realm. Bonus: capes are optional!

### Why Work at Anthropic? 
- Engage in daily research that might just change the world.
- Work in a vibrant environment that celebrates open communication (and sometimes even coffee spills, because that's creativity, right?).
- Learn with *Anthropic Academy*—a place to master the art of building with Claude. Think of it as Hogwarts for AI enthusiasts!

## What Our Customers Say! 🎤
Want to hear from our satisfied customers? We've got case studies and success stories galore! Discover how Claude has helped them *navigate* challenges like a trusty sidekick (possibly even more effective than a GPS!).

## Final Thoughts ✨
At Anthropic, we don't just build AI—we craft safe, benevolent, chatty companions that can make your life a whole lot smoother... one *Sonnet* at a time! So whether you're a prospective customer, investor, or recruit, come be a part of our *safe* adventure!

---

Got questions? Feel free to **Chat with Claude** or delve into our splendid **resources**. Remember, responsible AI is just a conversation away!

"# Welcome to Anthropic: Where AI Meets Safety with a Touch of Sass! 🎉\n\n## Who Are We? \nAt **Anthropic**, we create dynamic AI that serves humanity and put safety right at the forefront. Think of us as the dandy safety patrol officers of the AI world with a very clever, chatty companion named **Claude**! Whether you need support for coding, collaboration, or those downright tricky customer queries, Claude is here to serve up answers like a master chef—just without the cooking!\n\n## Meet Claude! 🤖\nClaude is our enchanting AI model that has gone through more evolutions than a Pokémon! Currently rocking the *3.7 Sonnet* version (take that, Shakespeare!), Claude can help with everything from research to product design. Plus, he’ll keep it safe while doing so, because let's face it, no one wants a rogue AI causing chaos in the digital library!\n\n## Company Culture: A Safe Haven for Big Thinkers 🧠✨\nAt Anthropic, we believe in **bold steps forward** mixed with intentional pauses. It’s 

In [80]:

stream_brochure("Earnix", "https://earnix.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://earnix.com/our-story/'}, {'type': 'careers page', 'url': 'https://earnix.com/our-story/careers/'}, {'type': 'solutions page', 'url': 'https://earnix.com/our-solutions/'}, {'type': 'customers page', 'url': 'https://earnix.com/our-customers/'}, {'type': 'contact page', 'url': 'https://earnix.com/contact/'}, {'type': 'insights page', 'url': 'https://earnix.com/insights-center/'}]}


# 🎉 Welcome to Earnix! 🍪

### Where Cookies Are Our Best Friends and Financial Operations Are *Strangely* Fun! 🎈

---

## **About Us**

At **Earnix**, we’re not just crunching numbers; we’re turning them into pure operational joy! 🚀 Our mission is to bring a sprinkle of *intelligence* to the insurance and banking sectors. Think of us as the fairy godmothers of financial operations — only instead of wands, we wield data analytics and operational excellence. ✨

---

## **Our Creamy Cookie Policy 🍪**

We believe in sharing… cookies! Yes, on our website, we use cookies to personalize your experience, manage ads, and analyze traffic. We promise we’re not tracking you to find out what TV shows you binge-watch late at night (but if we did, we’d totally recommend a good comedy).

- **Necessary Cookies:** Because what's a website without a bit of navigation? 
- **Preference Cookies:** For when you just can’t decide between English or a dash of French. 
- **Statistics Cookies:** Like a coach looking at game stats, we gather info to win the big games!

**Consent?** Check. 🚦

---

## **A Taste of Our Culture 🍰**

At Earnix, we thrive on collaboration, creativity, and occasionally pretending to be British while we sip tea. 🇬🇧 From our open office layout (where the only barrier is a friendly game of ping-pong) to our lively Friday social hours, we know that a good laugh is as important as good leadership. Join us if you’re ready to bring your A-game and a few cheesy jokes!

---

## **Who We Serve? 🎯**

We cater to the fabulous world of **insurance and banking**, providing them with intelligent operations that make their lives easier. Our clients range from small startups to financial giants, all looking for that secret sauce to make their business operations flow smoother than a jazz saxophonist on a Sunday morning. 🎷

---

## **Careers at Earnix: Come for the Cookies, Stay for the Culture! 🧑‍💼👩‍💼**

We’re always on the lookout for talent! Are you:
- Witty enough to make the HR team chuckle? 
- Wise enough to enlighten us on data analytics? 
- Creative enough to invent the next big thing?

If you answered “yes” to any of those questions, **welcome home!** Let us know if you want to join our team of rockstars where innovation meets fun! 🌟

---

## **In Summary... 🍾**

Whether you're a future employee, an investor with a penchant for high returns, or a potential partner ready to revolutionize the insurance and banking industries, **Earnix** is your ideal match! Let's team up to turn numbers into opportunities and have a good laugh while we're at it. 

**Join us on this wild ride! 🚀**

--- 

**Disclaimer:** No cookies (the digital kind) were harmed in the making of this brochure. Please accept all cookies responsibly. 🍪

"# 🎉 Welcome to Earnix! 🍪\n\n### Where Cookies Are Our Best Friends and Financial Operations Are *Strangely* Fun! 🎈\n\n---\n\n## **About Us**\n\nAt **Earnix**, we’re not just crunching numbers; we’re turning them into pure operational joy! 🚀 Our mission is to bring a sprinkle of *intelligence* to the insurance and banking sectors. Think of us as the fairy godmothers of financial operations — only instead of wands, we wield data analytics and operational excellence. ✨\n\n---\n\n## **Our Creamy Cookie Policy 🍪**\n\nWe believe in sharing… cookies! Yes, on our website, we use cookies to personalize your experience, manage ads, and analyze traffic. We promise we’re not tracking you to find out what TV shows you binge-watch late at night (but if we did, we’d totally recommend a good comedy).\n\n- **Necessary Cookies:** Because what's a website without a bit of navigation? \n- **Preference Cookies:** For when you just can’t decide between English or a dash of French. \n- **Statistics Cookies:

In [66]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}



# Welcome to Hugging Face! 🎉

**The AI Community Building the Future (and maybe your next coffee!)**

---

## 🌟 Meet Hugging Face

Looking for an AI platform that’s as friendly as a puppy and as innovative as sliced bread? Welcome aboard! Hugging Face is where the machine learning community gathers to create, collaborate, and caffeinate! With over **1 MILLION models** and even more datasets, we’re on a quest to build the future of machine learning (and life as we know it!)

### 🤖 Why Hugging Face?

- **Models Galore**: Whether you need AI for text, audio, images, or even 3D (yes, really!), we’ve got your back. Our ever-expanding library includes trending models like:
  - 🦜 *nvidia/parakeet-tdt-0.6b-v2* - Updated 9 days ago: A sure-fire way to get your parrots to finally talk!
  - 😻 *ACE-Step/ACE-Step-v1-3.5B* - Updated hours ago: Because music generation needs a little love too!

- **More Data Than You Can Shake a Stick At**: Our treasure trove includes 250k+ datasets that’ll make you feel like the festive king (or queen) of AI.

- **Collaboration Kingdom**: With **over 50,000 organizations** collaborating-us, from Meta to Microsoft, we say, “Why compete when we can create?”

---

## 🎉 Company Culture

At Hugging Face, we value **collaboration** over competition, **fun** over frowns, and **innovation** over monotony. Our workspace is filled with snacks, laughter, and the occasional cookie decorating contest. Yes, we do take our cookies seriously! 

We believe in as much sharing and caring as the internet allows – our *Open Source* values mean that *everyone's welcome* to join the party, including your Great Aunt Edna who just learned how to turn on her computer.

### 👔 Careers

Are you ready to join an environment where *brainstorming awards* are given for the most ridiculous ideas?  Dive into a career at Hugging Face where you can:
- Develop state-of-the-art ML models 
- Participate in community challenges that might involve fashionable AI avatars
- Improve your skills while being part of a team that knows how to *accept a hug* (virtually or in-person)!

### **Current Openings**: 
- Zen Master of Data (a.k.a Data Scientist)
- Machine Learning Wizard (a.k.a ML Engineer)
- Snacks Quality Control Specialist (We take our snacks very seriously!)

---

## 📞 Join the Hugging Face Family!

Whether you’re a customer looking to explore AI, an investor ready to ride the AI wave, or a recruit ready to shake things up, Hugging Face is your cozy corner in the tech universe.

**📧 Contact Us:**
Visit our *website* or slide into our DMs on LinkedIn, Twitter, or even Discord! (We love a good chat!)

Let’s build the future together. Just remember, in this world of machines - we promise to always keep our hugs human-sized!

**Hugs** 🤗,
The Hugging Face Team



"\n# Welcome to Hugging Face! 🎉\n\n**The AI Community Building the Future (and maybe your next coffee!)**\n\n---\n\n## 🌟 Meet Hugging Face\n\nLooking for an AI platform that’s as friendly as a puppy and as innovative as sliced bread? Welcome aboard! Hugging Face is where the machine learning community gathers to create, collaborate, and caffeinate! With over **1 MILLION models** and even more datasets, we’re on a quest to build the future of machine learning (and life as we know it!)\n\n### 🤖 Why Hugging Face?\n\n- **Models Galore**: Whether you need AI for text, audio, images, or even 3D (yes, really!), we’ve got your back. Our ever-expanding library includes trending models like:\n  - 🦜 *nvidia/parakeet-tdt-0.6b-v2* - Updated 9 days ago: A sure-fire way to get your parrots to finally talk!\n  - 😻 *ACE-Step/ACE-Step-v1-3.5B* - Updated hours ago: Because music generation needs a little love too!\n\n- **More Data Than You Can Shake a Stick At**: Our treasure trove includes 250k+ dataset

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>